In [1]:
import requests
import pandas as pd
from API_Key import api_key

pages = list(range(1,11))

popular = pd.DataFrame()

for page in pages:
    movieData = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={page}').json()['results'])
    popular = popular.append(movieData)


popular = popular[popular['adult'] == False]
popular

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,"[28, 12, 14]",566525,en,Shang-Chi and the Legend of the Ten Rings,Shang-Chi must confront the past he thought he...,5884.885,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,2021-09-01,Shang-Chi and the Legend of the Ten Rings,False,7.9,3428
1,False,/z3TFGLBUBF8pLkSO6ohkhMe4tMt.jpg,"[878, 28, 12]",580489,en,Venom: Let There Be Carnage,After finding a host body in investigative rep...,5797.863,/rjkmN1dniUHVYAtwuV3Tji7FsDO.jpg,2021-09-30,Venom: Let There Be Carnage,False,7.0,2452
2,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,"[28, 35, 80, 53]",512195,en,Red Notice,An Interpol-issued Red Notice is a global aler...,5515.987,/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg,2021-11-04,Red Notice,False,6.9,1263
3,False,/u5Fp9GBy9W8fqkuGfLBuuoJf57Z.jpg,"[12, 28, 53]",370172,en,No Time to Die,Bond has left active service and is enjoying a...,3366.389,/iUgygt3fscRoKWCV1d0C7FbM9TP.jpg,2021-09-29,No Time to Die,False,7.6,2075
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,"[16, 35, 10751]",585245,en,Clifford the Big Red Dog,As Emily struggles to fit in at home and at sc...,3116.768,/tY5jylU5wWaqRs8fmwkJBNzoyXB.jpg,2021-11-10,Clifford the Big Red Dog,False,7.8,182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,False,/5C8bfwglg91uZhc2fbfpSjNGamV.jpg,"[27, 9648]",591273,en,Fear Street: 1994,"After a series of brutal slayings, a teen and ...",178.596,/9J9Wy39ZjrVmfk7yMkulpcI5sy0.jpg,2021-06-28,Fear Street: 1994,False,6.8,1248
16,False,/4K57Vd78y3pMsAAfOdZYATTqnL3.jpg,"[28, 53, 80]",15092,en,Crank: High Voltage,Chelios faces a Chinese mobster who has stolen...,177.734,/tzTC4EEvF0OPL63frEiogxL2T8M.jpg,2009-04-16,Crank: High Voltage,False,6.0,2132
17,False,/7vDOtDCVuwp84lvdLp7IxheNWWr.jpg,"[28, 53]",38234,en,Undisputed III: Redemption,"Russian inmate Boyka, now severely hobbled by ...",177.714,/g8KB77SPA7SyU8eid6TAEpt9skU.jpg,2010-05-22,Undisputed III: Redemption,False,7.5,571
18,False,/1MxdxiUvKCbuKDoeiuq668eIKb8.jpg,"[16, 28, 12, 35, 10751, 14, 878]",834404,en,Teen Titans Go! See Space Jam,"The Teen Titans are visited by the Nerdlucks, ...",176.341,/rMoVOCw6DF3zC6hS76ZhQMWiXNX.jpg,2021-06-20,Teen Titans Go! See Space Jam,False,7.0,145


In [2]:
popular_features = popular[['id','title','genre_ids','release_date','popularity','vote_average','vote_count']]
popular_features = popular_features.rename(columns={
                                        "id":"movie_id",
                                        "popularity":"movie_popularity"
                                        })

In [3]:
id_list = popular_features['movie_id'].tolist()
revenue_list = []
budget_list = []

for movie in id_list:
    x = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['revenue']
    y = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['budget']
    revenue_list.append(x)
    budget_list.append(y)

revenue_df = pd.DataFrame(
    {'movie_id':id_list,
    'budget': budget_list,
    'revenue':revenue_list
    })

revenue_df.reset_index(drop = True)

,movie_id,budget,revenue
0,566525,150000000,430238384
1,580489,110000000,454000000
2,512195,200000000,0
3,370172,242000000,734000000
4,585245,0,22000000
...,...,...,...
195,591273,0,0
196,15092,20000000,34560577
197,38234,3000000,0
198,834404,0,0


In [4]:

Movie_Data = popular_features.merge(revenue_df, on='movie_id', how='inner')
Movie_Data.count()

movie_id            200
title               200
genre_ids           200
release_date        200
movie_popularity    200
vote_average        200
vote_count          200
budget              200
revenue             200
dtype: int64

In [5]:
ratings = []
x = 0
for movie_id in Movie_Data['movie_id']:
    movie_rating = requests.get(f'http://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=release_dates').json()['release_dates']['results']
    for rating in movie_rating:
        if rating['iso_3166_1'] == 'US':
            try:
                ratings.append(rating['release_dates'][1]["certification"])
                x=1
            except IndexError:
                x=1
                ratings.append(rating['release_dates'][0]["certification"])
    if x == 0:
        ratings.append('')
    x=0

In [6]:
Movie_Data['rating'] = ratings
Movie_Data

,movie_id,title,genre_ids,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating
0,566525,Shang-Chi and the Legend of the Ten Rings,"[28, 12, 14]",2021-09-01,5884.885,7.9,3428,150000000,430238384,PG-13
1,580489,Venom: Let There Be Carnage,"[878, 28, 12]",2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13
2,512195,Red Notice,"[28, 35, 80, 53]",2021-11-04,5515.987,6.9,1263,200000000,0,PG-13
3,370172,No Time to Die,"[12, 28, 53]",2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13
4,585245,Clifford the Big Red Dog,"[16, 35, 10751]",2021-11-10,3116.768,7.8,182,0,22000000,PG
...,...,...,...,...,...,...,...,...,...,...
195,591273,Fear Street: 1994,"[27, 9648]",2021-06-28,178.596,6.8,1248,0,0,R
196,15092,Crank: High Voltage,"[28, 53, 80]",2009-04-16,177.734,6.0,2132,20000000,34560577,R
197,38234,Undisputed III: Redemption,"[28, 53]",2010-05-22,177.714,7.5,571,3000000,0,R
198,834404,Teen Titans Go! See Space Jam,"[16, 28, 12, 35, 10751, 14, 878]",2021-06-20,176.341,7.0,145,0,0,NR


In [7]:
filtered_table = Movie_Data[Movie_Data['budget']!= 0]
filtered_table = filtered_table[filtered_table['revenue'] != 0]
filtered_movies = filtered_table[filtered_table['rating'] != '']
filtered_movies

,movie_id,title,genre_ids,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating
0,566525,Shang-Chi and the Legend of the Ten Rings,"[28, 12, 14]",2021-09-01,5884.885,7.9,3428,150000000,430238384,PG-13
1,580489,Venom: Let There Be Carnage,"[878, 28, 12]",2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13
3,370172,No Time to Die,"[12, 28, 53]",2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13
11,524434,Eternals,"[28, 12, 878]",2021-11-03,1746.171,7.1,1170,200000000,336000000,PG-13
12,744275,After We Fell,"[10749, 18]",2021-09-01,1710.038,7.2,956,14000000,19000000,R
...,...,...,...,...,...,...,...,...,...,...
178,303857,Dragon Ball Z: Resurrection 'F',"[28, 16, 878]",2015-04-18,192.039,6.8,1246,5000000,61768190,NR
181,565028,Candyman,"[27, 53]",2021-08-25,188.391,6.3,615,25000000,77389310,R
182,400160,The SpongeBob Movie: Sponge on the Run,"[10751, 16, 14, 12, 35]",2020-08-14,186.863,7.7,2351,60000000,4700000,PG
187,72545,Journey 2: The Mysterious Island,"[12, 28, 878]",2012-01-19,185.102,6.1,3312,79000000,355692760,PG


In [8]:
filtered_movies.count()

movie_id            74
title               74
genre_ids           74
release_date        74
movie_popularity    74
vote_average        74
vote_count          74
budget              74
revenue             74
rating              74
dtype: int64

In [9]:

titleNid = filtered_movies[['movie_id','title']]

titleNid = titleNid.reset_index(drop = True)

titleNid.to_csv('../Tables/titleNid.csv',index = False)

filtered_movies.to_csv('../Tables/Movie_Data.csv', index = False)

In [10]:
genre = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US').json()['genres']
genre_df = pd.DataFrame(genre)
genre_df
genre_df.to_csv('../Tables/Genres.csv',index = False)